# Tugas 1b: Lakukan Pembuatan Teks menggunakan perintah yang menyertakan Konteks

Dalam <i>notebook</i> ini, Anda akan belajar cara menghasilkan respons email kepada pelanggan yang tidak puas dengan kualitas layanan pelanggan yang diterima dari teknisi dukungan pelanggan. Anda akan memberikan konteks tambahan pada model dengan memasukkan konten email sebenarnya yang diterima dari pelanggan yang tidak puas.

Anda akan menambahkan kompleksitas lain dengan bantuan PromptTemplates untuk memanfaatkan kerangka kerja LangChain untuk kasus penggunaan serupa. PromptTemplates memungkinkan Anda membuat <i>shell</i> generik yang dapat diisi dengan informasi nanti dan mendapatkan <i>output</i> model berdasarkan skenario yang berbeda.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) adalah kerangka kerja untuk mengembangkan aplikasi yang didukung model bahasa. Aspek utama dari kerangka kerja ini memungkinkan kami meningkatkan Model Bahasa Besar dengan menyatukan berbagai komponen untuk menciptakan kasus penggunaan lanjutan.

Karena konteks tambahan dalam perintah, konten yang dihasilkan dalam <i>notebook</i> ini memiliki kualitas dan relevansi yang jauh lebih baik daripada konten yang diproduksi sebelumnya melalui perintah <i>zero-shot</i>. Perintah yang digunakan dalam <i>notebook</i> ini membuat templat perintah LangChain kustom untuk menambahkan konteks ke permintaan pembuatan teks.

#### Skenario
Anda adalah Bob, Manajer Layanan Pelanggan di AnyCompany. Beberapa pelanggan Anda tidak puas dengan layanan pelanggan dan memberikan umpan balik negatif terhadap layanan yang diberikan oleh teknisi dukungan pelanggan. Sekarang, Anda ingin merespons pelanggan tersebut dengan meminta maaf atas layanan yang buruk dan mendapatkan kembali kepercayaan mereka. Anda memerlukan bantuan Model Bahasa Besar (LLM) untuk membuat sejumlah besar email yang ramah manusia dan disesuaikan dengan sentimen pelanggan dari korespondensi email sebelumnya.

Dalam skenario ini, Anda dapat memanfaatkan kekuatan PromptTemplates LangChain untuk membuat <i>shell</i> generik untuk menghasilkan tanggapan email yang dipersonalisasi berdasarkan email pelanggan sebelumnya. PromptTemplate akan menggabungkan konten email asli pelanggan agar LLM dapat memahami konteks dan sentimen, dan kemudian menghasilkan respons yang relevan dan disesuaikan.

## Tugas 1b.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tugas 1b.2: Menginvokasi Model LLM Bedrock

Dalam tugas ini, Anda akan membuat instans kelas Bedrock dari llms. Instans ini mengharapkan `model_id` yang merupakan Amazon Resource Name (ARN) dari model yang tersedia di Amazon Bedrock.

Secara opsional, Anda dapat meneruskan klien boto3 yang dibuat sebelumnya serta beberapa `model_kwargs` yang dapat menyimpan parameter seperti `temperature`, `top_p`, `max_token_count`, atau `stop_sequences` (informasi lebih lanjut tentang parameter dapat dipelajari di konsol Amazon Bedrock).

Lihat [dokumentasi] (https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) untuk ID model pembuatan teks yang tersedia di bagian Amazon Bedrock.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**Catatan: ** Model yang berbeda mendukung `model_kwargs` yang berbeda.

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## Tugas 1b.3: Membuat templat perintah khusus LangChain

Dalam tugas ini, Anda akan membuat templat untuk perintah bahwa Anda dapat meneruskan variabel input yang berbeda pada setiap proses. Hal ini berguna ketika Anda harus membuat konten dengan variabel input berbeda yang mungkin diambil dari basis data.

Pada tugas sebelumnya, kami melakukan <i>hardcode</i> perintah. Mungkin ada beberapa pelanggan yang mengirimkan umpan balik negatif serupa dan sekarang Anda ingin menggunakan setiap email pelanggan tersebut dan membalasnya dengan permintaan maaf, tetapi Anda juga ingin menjaga agar responsnya sedikit dipersonalisasi. Di sel berikut, Anda akan mempelajari cara membuat `PromptTemplate` untuk mencapai pola ini.

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**Catatan: ** Anda dapat mengabaikan peringatan dan melanjutkan ke sel berikutnya dengan aman.

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

Anda telah berhasil mempelajari bahwa memanggil LLM tanpa konteks apa pun mungkin tidak memberikan hasil yang diinginkan. Dengan menambahkan konteks dan menggunakan templat perintah lebih lanjut untuk membatasi <i>output</i> dari LLM, Anda berhasil mendapatkan <i>output </i>yang diinginkan.

### Cobalah sendiri
- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini.
- Kembali ke sesi lab dan lanjutkan dengan **Tugas 2**.